# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
J. G. Lobos  ->  J. G. Lobos  |  ['J. G. Lobos']
J. Li  ->  J. Li  |  ['J. Li']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']


S. Torniamenti  ->  S. Torniamenti  |  ['S. Torniamenti']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Shi  ->  J. Shi  |  ['J. Shi']
Arxiv has 94 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.11778


extracting tarball to tmp_2510.11778...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2510.11778/main_aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'figures/fig_noise' from 'tmp_2510.11778/figures/fig_noise.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/tab_agemass_assoc' from 'tmp_2510.11778/tables/tab_agemass_assoc.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.

...

/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Plot/smax_pop_bin_Z.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\textwidth]{Plot/smax_pop_bin_Z.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 95 bibliographic references in tmp_2510.12465/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2510.12518
extracting tarball to tmp_2510.12518...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.12627
extracting tarball to tmp_2510.12627...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.12658
extracting tarball to tmp_2510.12658...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.11778-b31b1b.svg)](https://arxiv.org/abs/2510.11778) | **The PHANGS-MUSE/HST-Halpha Nebulae Catalogue**  |
|| A. T. Barnes, et al. -- incl., <mark>K. Kreckel</mark>, <mark>J. G. Lobos</mark>, <mark>J. Li</mark>, <mark>J. Neumann</mark> |
|*Appeared on*| *2025-10-15*|
|*Comments*| *16 (+10) pages, 14 (+7) figures, 1 (+7) tables, accepted for publication in A&A, full catalogue available via CDS*|
|**Abstract**|            We present the PHANGS-MUSE/HST-Halpha nebulae catalogue, comprising 5177 spatially resolved nebulae across 19 nearby star-forming galaxies (< 20 Mpc), based on high-resolution Halpha imaging from HST, homogenised to a fixed 10 pc resolution and sensitivity. Combined with MUSE spectroscopy, this enables robust classification of 4882 H II regions and separation of planetary nebulae and supernova remnants. Electron densities for 2544 H II regions are derived using [S II] diagnostics, and nebular sizes measured via circularised radii and second moments yield a median of 20 pc, extending to sub-parsec scales. A structural complexity score traces substructure, showing that about a third of regions are H II complexes, with a higher fraction in galaxy centres. A luminosity-size relation calibrated from the HST sample is applied to 30,790 MUSE nebulae, recovering sizes down to 1 pc. Observed sizes exceed classical Stromgren radii, implying typical volume filling factors of 0.22. We associate 3349 H II regions with stellar populations from PHANGS-HST, finding median ages of 3 Myr and masses of 4-5 log(Msun). The dataset provides a detailed, spatially resolved link between nebular structure and ionising sources, serving as a benchmark for future studies of feedback, diffuse ionised gas, and star formation regulation in the interstellar medium. The full catalogue is made publicly available in machine-readable format.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.12465-b31b1b.svg)](https://arxiv.org/abs/2510.12465) | **Optically thick winds of very massive stars suppress intermediate-mass black hole formation**  |
|| <mark>S. Torniamenti</mark>, et al. |
|*Appeared on*| *2025-10-15*|
|*Comments*| *10 pages, 7 figures. Comments welcome*|
|**Abstract**|            Intermediate-mass black holes (IMBHs) are the link between stellar-mass and supermassive black holes. Gravitational waves have started unveiling a population of IMBHs in the $\sim 100-300 \, \mathrm{M_{\odot}}$ range. Here, we investigate the formation of IMBHs from very massive stars (VMSs, $>100\,{} \mathrm{M_{\odot}}$). We calculate new VMS models that account for the transition from optically thin to optically thick winds, and study how this enhanced mass loss affects IMBH formation and the black hole mass function at intermediate and high metallicity ($Z=10^{-4}-0.02$). We show that optically thick winds suppress the formation of IMBHs from direct VMS collapse at metallicities $Z>0.001$, one order of magnitude lower than predicted by previous models. Our models indicate that the stellar progenitors of GW231123 must have had a metallicity $Z<0.002$, if the primary black hole formed via direct VMS collapse.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.11785-b31b1b.svg)](https://arxiv.org/abs/2510.11785) | **Spin and Obliquity Distributions of Low-mass Planets Shaped by Dynamical Instability**  |
|| D. Wang, <mark>J. Li</mark>, D. Lai |
|*Appeared on*| *2025-10-15*|
|*Comments*| *10 pages, 6 figures, 1 table, submitted to ApJ*|
|**Abstract**|            Exoplanetary systems hosting multiple low-mass planets are thought to have experienced dynamical instability, during which planet-planet collisions and mergers occur; these collisions can impart substantial amount of angular momentum to the merger remnants, changing the obliquities of the resulting planets significantly. In this work, we carry out a series of $N$-body experiments to investigate the spin magnitude $(|\vec{S}|)$ and obliquity $(\theta_{\rm SL})$ distributions of low-mass exoplanets that have gone through planetary collisions. In our fiducial super-Earth (with $m=3M_{\oplus}$, $R=1.3R_{\oplus}$) and mini-Neptune systems (with $m=9M_{\oplus}$, $R=2.5R_{\oplus}$), the collision products follow a nearly uniform distribution in $\cos{\theta_{\rm SL}}$ and the spin-magnitude distribution is approximately linear in $|\vec{S}|$. Parameter studies and theoretical analysis show that increasing planetary radii or masses, or decreasing the initial planet-planet mutual inclinations, tend to polarize the obliquity distribution toward alignment or anti-alignment (i.e., excess probability near $\cos{\theta_{\rm SL}}=\pm1$). Experiments with initially two-planet and three-planet systems produce qualitatively similar outcomes, suggesting that the trends in this study may generalize to systems with higher planetary multiplicities.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.12162-b31b1b.svg)](https://arxiv.org/abs/2510.12162) | **CCAT: Optical Responsivity, Noise, and Readout Optimization of KIDs for Prime-Cam**  |
|| E. Gazda, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-15*|
|*Comments*| **|
|**Abstract**|            The Prime-Cam instrument of the Fred Young Submillimeter Telescope (FYST) at the CCAT Observatory will conduct sensitive millimeter to submillimeter surveys for a range of astrophysical and cosmological sciences. Prime-Cam will use kinetic inductance detectors (KIDs) sensitive to multiple frequency bands spanning 280--850 GHz. With over 100,000 sensors under development, these KID arrays will soon form the largest submillimeter focal plane ever built. With fixed microwave tones probing amplitude and phase modulations in the KIDs due to incoming radiation, challenges arise in determining the optimal readout settings, especially under varying atmospheric loading. Realizing the science goals of FYST requires operating the detectors at optimal performance and determining accurate responsivities, which depend on readout tone placement and power. To address these challenges, we present laboratory measurements of sample pixels from the 280 GHz TiN and Al arrays using a blackbody cold load to simulate observing conditions. These measurements probe detector responsivity and noise across varying optical loading, tone power, and tone placement, providing the foundation to guide in situ calibration and operation of the $>$100,000 KIDs. We characterize detector sensitivity via the Noise Equivalent Power (NEP) as a function of readout tone power and placement, and measure the impact of detuning due to varying optical power on the NEP. Our test setup and methodology will inform the commissioning of Prime-Cam, in situ detector calibration procedures, the cadence of probe tone resetting, and potential design refinements for future arrays, supporting FYST's planned first light in 2026.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.12235-b31b1b.svg)](https://arxiv.org/abs/2510.12235) | **A census of quiescent galaxies across $0.5 < z < 8$ with JWST/MIRI: Mass-dependent number density evolution of quiescent galaxies in the early Universe**  |
|| T. Yang, et al. -- incl., <mark>L. Xie</mark>, <mark>Y. Wu</mark> |
|*Appeared on*| *2025-10-15*|
|*Comments*| *17 pages, 5 figures, submitted to ApJL*|
|**Abstract**|            JWST observations reveal numerous quiescent galaxies (QGs) at high redshift ($z \sim 4-8$), challenging models of early galaxy formation and quenching. Accurate number density estimates are crucial for comparison with theory but remain uncertain. We systematically study QGs at $0.5 < z < 8$ using a mass-complete sample from the JWST/PRIMER survey with deep NIRCam and MIRI imaging. The MIRI data, probing rest-frame near-infrared at $z \sim 3-8$, are vital for robust stellar mass measurement and QG identification. We find that nearly all photometrically selected, point-like QG candidates located in the UVJ QG region are actually "Little Red Dots", for which the UVJ colors were wrongly estimated due to inaccurate photometric redshift estimation. MIRI reduces significantly contamination to high-mass QGs from star-forming galaxies, yielding lower number densities than previous studies. The evolution of QG number density is strongly mass-dependent. The density of high-mass QGs ($\log (M_{\star}/M_{\odot}) > 10.3$) decreases rapidly from $n = 1\times10^{-5}~\mathrm{Mpc^{-3}}$ at $z=3-4$ to $n=2\times10^{-6}~\mathrm{Mpc^{-3}}$ at $z = 4-5$, becoming negligible ($n \lesssim 10^{-6}~\mathrm{Mpc^{-3}}$ ) at $z > 5$. Conversely, low-mass QGs ($9<\log (M_{\star}/M_{\odot})<10.3$) maintain a nearly constant number density ($n\sim3\times10^{-6}~\mathrm{Mpc^{-3}}$) across $z = 4-8$. This suggests low-mass QGs at $z > 4$ are likely temporarily quenched, akin to mini-quenched galaxies. Comparison with major hydrodynamical and semi-analytical models shows most underestimate high-mass QG densities at $z>4$ and fail to reproduce the constant low-mass QG density at $z>5$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.12518-b31b1b.svg)](https://arxiv.org/abs/2510.12518) | **Widespread Hot Molecular Gas Heated by Shear-induced Turbulence in the Galactic Center**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2025-10-15*|
|*Comments*| *21 pages, 7 figures. Accepted by ApJ*|
|**Abstract**|            We observed NH3 metastable inversion lines from (3, 3) to (18, 18) toward G0.66-0.13 in the Galactic center with the Shanghai Tianma 65m radio telescope and Yebes 40 m telescope. Highly-excited lines of NH3 (17, 17), (18, 18) were detected in emission for the first time in the interstellar medium, with upper energy levels up to 3100 K. Mapping observations reveal widespread hot molecular gas traced by NH3 (13, 13) toward G0.66-0.13. The rotation temperatures of hot gas traced by NH3 exceed 400 K, which amounts to five percent of the total NH3 in the Galactic Center. Hot gas (>400 K) and warm gas (100-140 K) are found in distinct clumps, with the hot gas located at the interfacing regions between different warm clouds. The theory of intermittency in turbulence reproduces the complex temperature structure in the central molecular zone, especially the hot gas observed here. The results presented here demonstrate that turbulence heating dominates the heating of the molecular gas in the Central Molecular Zone, while the turbulence is induced by the shear-motion of molecular clouds under the gravitational potential of the nuclear star clusters and the supermassive black hole. Our results suggest that shear-induced turbulence heating could be a widespread factor influencing galactic evolution.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.12627-b31b1b.svg)](https://arxiv.org/abs/2510.12627) | **Alleviating the $H_0$ tension through new interacting dark energy model in light of DESI DR2**  |
|| Y.-M. Zhang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-15*|
|*Comments*| *12 pages, 4 figures*|
|**Abstract**|            The $H_0$ tension has become one of the most significant challenges in modern cosmology. The recent DESI DR2 data has shown a significant preference for dynamical dark energy, yet this has further exacerbated the $H_0$ tension. In this work, we explore the potential of new interacting dark energy models ($\widetilde{\Lambda}$CDM and $e\widetilde{\Lambda}$CDM) to alleviate the $H_0$ tension. We perform observational constraints using the latest baryon acoustic oscillation data from DESI DR2, cosmic microwave background (CMB) data from Planck and Atacama Cosmology Telescop, and type Ia supernova data from DESY5 and PantheonPlus, as well as the SH0ES data. From our analysis, we observe the dynamical scale parameter of the cosmological constant, $\delta_{\Lambda} = -0.410^{+0.140}_{-0.120}$, in the $e\widetilde{\Lambda}$CDM model using the CMB+DESI+SH0ES data, which deviates from $\Lambda$CDM at the $3.2\sigma$ level. Due to the anti-correlation between $\delta_{\Lambda}$ and $H_0$, a negative $\delta_{\Lambda}$ results in a higher inferred $H_0$. Consequently, we obtain $H_0 = 71.90 \pm 1.00~\mathrm{km\,s^{-1}\,Mpc^{-1}}$, reducing the $H_0$ tension to $0.8\sigma$. Even without SH0ES, the CMB+DESI data alone still alleviate the $H_0$ tension to $1.7\sigma$. Overall, the $e\widetilde{\Lambda}$CDM model not only deviates from the $\Lambda$CDM model but also demonstrates a significant capability to alleviate the $H_0$ tension.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.12658-b31b1b.svg)](https://arxiv.org/abs/2510.12658) | **Enigmatic centi-SFU and mSFU nonthermal radio transients detected in the middle corona**  |
|| S. Mondal, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-10-15*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            Decades of solar coronal observations have provided substantial evidence for accelerated particles in the corona. In most cases, the location of particle acceleration can be roughly identified by combining high spatial and temporal resolution data from multiple instruments across a broad frequency range. In almost all cases, these nonthermal particles are associated with quiescent active regions, flares, and coronal mass ejections (CMEs). Only recently, some evidence of the existence of nonthermal electrons at locations outside these well-accepted regions has been found. Here, we report for the first time multiple cases of transient nonthermal emissions, in the heliocentric range of $\sim 3-7R_\odot$, which do not have any obvious counterparts in other wavebands, like white-light and extreme ultra-violet. These detections were made possible by the regular availability of high dynamic range low-frequency radio images from the Owens Valley Radio Observatory's Long Wavelength Array. While earlier detections of nonthermal emissions at these high heliocentric distances often had comparable extensions in the plane-of-sky, they were primarily been associated with radio CMEs, unlike the cases reported here. Thus, these results add on to the evidence that the middle corona is extremely dynamic and contains a population of nonthermal electrons, which is only becoming visible with high dynamic range low-frequency radio images.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.11778/./figures/maps_hstmuse_ngc1566_zoom_compressed.png', 'tmp_2510.11778/./figures/maps_hstmuse_ngc1566_zoom_complex.png', 'tmp_2510.11778/./figures/maps_hstmuse_ngc1566_compressed.png']
copying  tmp_2510.11778/./figures/maps_hstmuse_ngc1566_zoom_compressed.png to _build/html/
copying  tmp_2510.11778/./figures/maps_hstmuse_ngc1566_zoom_complex.png to _build/html/
copying  tmp_2510.11778/./figures/maps_hstmuse_ngc1566_compressed.png to _build/html/
exported in  _build/html/2510.11778.md
    + _build/html/tmp_2510.11778/./figures/maps_hstmuse_ngc1566_zoom_compressed.png
    + _build/html/tmp_2510.11778/./figures/maps_hstmuse_ngc1566_zoom_complex.png
    + _build/html/tmp_2510.11778/./figures/maps_hstmuse_ngc1566_compressed.png
found figures ['tmp_2510.12465/./Plot/MBH_Z_mesa_parsec_v_log.png', 'tmp_2510.12465/./Plot/m1_merged_bin_Z.png', 'tmp_2510.12465/./Plot/mass_evolution.png']
copying  tmp_2510.12465/./Plot/MBH_Z_mesa_parsec_v_log.png to _build/html/
copyin

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\HII}{\ion{H}{II}}$
$\newcommand{\hii}{\ion{H}{II}}$
$\newcommand{\Ha}{H{\alpha}}$
$\newcommand{\kms}{\rm km s^{-1}}$
$\newcommand{\SIIa}{[\ion{S}{II}]\ensuremath{\lambda6716}}$
$\newcommand{\SIIb}{[\ion{S}{II}]\ensuremath{\lambda6731}}$
$\newcommand{\SII}{[\ion{S}{II}]\ensuremath{\lambda6716{+}\lambda6731}}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\nii}{[N \textsc{ii}]}$
$\newcommand{\sii}{[S \textsc{ii}]}$
$\newcommand{\oi}{[O \textsc{i}]}$
$\newcommand{\niion}{[N \textsc{i}]}$
$\newcommand{\hei}{[He \textsc{i}]}$
$\newcommand{\siii}{[S \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\Mgb}{Mg\textit{b}}$
$\newcommand{\NaI}{Na \textsc{I}}$
$\newcommand{\ha}{\mbox{\mathrm{H}\alpha}}$
$\newcommand{\hb}{\mbox{\mathrm{H}\beta}}$
$\newcommand{\Rsii}{\ensuremath{R_{\sii}}}$
$\newcommand{\xx}[1]{\textcolor{black}{ #1}}$
$\newcommand{\Csimple}{\mathcal{C}_{\mathrm{simp}}}$
$\newcommand{\Cintermediate}{\mathcal{C}_{\mathrm{inter}}}$
$\newcommand{\Ccomplex}{\mathcal{C}_{\mathrm{comp}}}$
$\newcommand{\fontcolor}{black}$
$\newcommand$
$\newcommand\kms{{\rm km s^{-1}}}$
$\newcommand\Kkms{{\rm K km s^{-1}}}$
$\newcommand\msun{{\rm M_\odot}}$
$\newcommand\Lsol{{\rm L_\odot}}$
$\newcommand\micron{\hbox{\upmum}}$
$\newcommand\solyr{{\rm M_\odot yr^{-1}}}$
$\newcommand\dyncm{{\rm dyn cm^{-2}}}$
$\newcommand\ergs{{\rm  erg s^{-1}}}$
$\newcommand\cmsq{{\rm cm^{-2}}}$
$\newcommand\cmcb{{\rm cm^{-3}}}$
$\newcommand\gcmsq{{\rm g cm^{-2}}}$
$\newcommand\gcmcb{{\rm g cm^{-3}}}$
$\newcommand\Kcmcb{{\rm K cm^{-3}}}$
$\newcommand\Lha{\ensuremath{L_{\mathrm{H}\alpha}}}$
$\newcommand\Lbol{\ensuremath{L_\mathrm{bol}}}$
$\newcommand\Lbolimf{\ensuremath{L_\mathrm{bol,imf}}}$
$\newcommand\Te{\ensuremath{T_\mathrm{e}}}$
$\newcommand\ne{\ensuremath{n_\mathrm{e}}}$
$\newcommand\ptot{\ensuremath{P_\mathrm{tot}}}$
$\newcommand\pdir{\ensuremath{P_\mathrm{rad}}}$
$\newcommand\ptherm{\ensuremath{P_\mathrm{therm}}}$
$\newcommand\pwind{\ensuremath{P_\mathrm{wind}}}$
$\newcommand\pde{\ensuremath{P_\mathrm{de}}}$
$\newcommand\pdess{\ensuremath{\langle P_\mathrm{de,\theta pc} \rangle_\mathrm{1kpc}}}$
$\newcommand\pmax{\ensuremath{P_\mathrm{max}}}$
$\newcommand\pmin{\ensuremath{P_\mathrm{min}}}$
$\newcommand\Ha{\textup{H\ensuremath{\alpha}}}$
$\newcommand\Hb{\textup{H\ensuremath{\beta}}}$
$\newcommand\Nmuseall{30,790}$
$\newcommand\Nmuse{25,910}$
$\newcommand\Nmusehii{19,528}$
$\newcommand\Nhst{5177}$
$\newcommand\Nhstnoflags{5467}$
$\newcommand\Nhstsimple{3386}$
$\newcommand\Nhstintermediate{1169}$
$\newcommand\Nhstcomplex{622}$
$\newcommand\Nhii{4882}$
$\newcommand\Nne{2544}$
$\newcommand\Nassoc{3349}$
$\newcommand\NTe{779}$
$\newcommand\NTefrac{14\%}$
$\newcommand\Nsnr{556}$
$\newcommand\Ncols{241}$</div>



<div id="title">

# The PHANGS-MUSE/HST-H$\alpha$ Nebulae Catalogue: Parsec-Scale Resolved Structure, Physical Conditions, and Stellar Associations across Nearby Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.11778-b31b1b.svg)](https://arxiv.org/abs/2510.11778)<mark>Appeared on: 2025-10-15</mark> -  _16 (+10) pages, 14 (+7) figures, 1 (+7) tables, accepted for publication in A&A, full catalogue available via CDS_

</div>
<div id="authors">

A. T. Barnes, et al. -- incl., <mark>K. Kreckel</mark>, <mark>J. G. Lobos</mark>, <mark>J. Li</mark>, <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** We present the PHANGS-MUSE/HST-H $\alpha$ nebulae catalogue, comprising $\Nhst$ spatially resolved nebulae across 19 nearby star-forming galaxies ( $D<20$ Mpc), based on high-resolution H $\alpha$ imaging from HST, homogenised to a fixed (10 pc) physical resolution and sensitivity. Combined with MUSE integral field spectroscopy, this enables robust classification of $\Nhii$ $\ion{H}{ii}$ regions and the separation of planetary nebulae and supernova remnants. We derive electron densities for $\Nne$ $\ion{H}{ii}$ regions using [ S ii ] diagnostics and adopt direct or representative electron temperatures for consistent physical characterisation. Nebular sizes are measured using circularised radii and intensity-weighted second moments, yielding a median radius of approximately 20 pc and extending down to (sub)-parsec (deconvolved) radii. A structural complexity score is introduced via hierarchical segmentation to trace substructure, highlighting that around a third of the regions are $\hii$ complexes containing several individual clusters/bubbles, with an increased fraction of these regions in galactic centres. A luminosity–size relation, calibrated using the resolved HST sample, is applied to $\Nmuseall$ MUSE nebulae, allowing the recovery of nebular sizes down to $\sim$ 1 pc and providing statistical completeness beyond the HST detection limit. Comparisons with classical Strömgren radii indicate that observed sizes are systematically larger, corresponding to typical volume filling factors with a \textcolor{\fontcolor}{median of $\epsilon \sim 0.22$ (10th–90th percentile 0.06–0.78), with larger regions exhibiting progressively lower values.} We associate $\Nassoc$ $\ion{H}{ii}$ regions with stellar populations from the PHANGS-HST association catalogue, finding median ages of $\sim$ 3 Myr and typical stellar masses of around $10^4$ – $10^5$ M $_\odot$ , supporting the link between ionised nebular and young stellar populations. We also assess the impact of diffuse ionised gas on emission-line diagnostics and, after removing confirmed supernova remnants, find no strong variation in line ratios with nebular resolution, indicating minimal systematic bias in the MUSE catalogue. This dataset establishes a detailed, spatially resolved connection between nebular structure and ionising sources, and provides a benchmark for future studies of feedback, DIG contributions, and star formation regulation in the ISM, especially in combination with matched high-resolution observations. \textcolor{\fontcolor}{The full catalogue is made publicly available in machine-readable format.}

</div>

<div id="div_fig1">

<img src="tmp_2510.11778/./figures/maps_hstmuse_ngc1566_zoom_compressed.png" alt="Fig12" width="100%"/>

**Figure 12. -** **Example of a $\hii$ complex region in NGC 1566 (see \S \ref{subsec_complexity**).}(_upper left panel_) MUSE observations and (_upper right_) HST $\Ha$ observations that are smoothed to a physical scale of 10 pc and with a fixed noise level (see \S\ref{subsec_DataHomogenization}). Each of these are overlaid with contours showing the boundary of the sources identified in the MUSE (white) and HST (black) observations. (_lower left panel_) A map of the complexity score for the region (see \S\ref{subsec_structure}). (_lower right panel_) We show the nebula (orange contours) and 32 pc NUV-identified  association (\citealp{Larson2023}; cyan contour) overlaid on a HST filter red (F814W) green (F555W) blue (F438W+F336W) image (see \citealp{Lee2022}). See Fig. \ref{fig_maps_zoom_complex} for additional examples of simple, intermediate and complex nebulae within NGC 1566. (*fig_maps_zoom_single*)

</div>
<div id="div_fig2">

<img src="tmp_2510.11778/./figures/maps_hstmuse_ngc1566_zoom_complex.png" alt="Fig16" width="100%"/>

**Figure 16. -** **Example of several regions within the nebula catalogues towards NGC 1566.** The MUSE (_first row_) and HST (_second row_) observations, overlaid with contours showing the boundary of each source in the respective observations. (_third row_) We show a map of the complexity score for each region. (_fourth row_) We show the nebula (red contours) and 32 pc NUV-identified stellar association (\citealp{Larson2023}; blue contour) overlaid on a HST filter red (F814W) green (F555W) blue (F438W+F336W) image (see \citealp{Lee2022}). (*fig_maps_zoom_complex*)

</div>
<div id="div_fig3">

<img src="tmp_2510.11778/./figures/maps_hstmuse_ngc1566_compressed.png" alt="Fig11" width="100%"/>

**Figure 11. -** **Overview of the nebula catalogues towards NGC 1566.**(_Upper left_) The background colour scale shows the H$\alpha$ emission from the PHANGS-MUSE observations  ([Emsellem, Schinnerer and Santoro 2022]()) , overlaid with contours showing the boundaries of the sources identified in the PHANGS-MUSE Nebula Catalogue \citep[][see also \citealp{Santoro2022}]{Groves2023}. White boxes indicate the positions of the regions shown in Figs. \ref{fig_maps_zoom_single}(solid lines) and  \ref{fig_maps_zoom_complex}(dashed lines). (_Upper centre_) The background colour scale shows the H$\alpha$ emission from the PHANGS-HST H$\alpha$ observations  ([Chandar, Barnes and Thilker 2025]()) , overlaid with contours showing the boundaries of the sources identified in this work in the PHANGS-HST Nebula Catalogue. (_Bottom left_) The PHANGS-MUSE Nebula Catalogue masks and (_bottom centre_) PHANGS-HST Nebula Catalogue masks are shown with the same colour scale, where the colours denote the region ID. (_Right_) Composite of separate exposures acquired with JWST using the NIRCam and MIRI instruments  ([Lee, Sandstrom and Leroy 2023](), [Williams, Lee and Larson 2024]()) , and the HST using the ACS/WFC instrument  ([Lee, Whitmore and Thilker 2022]()) . For the JWST part of the image, the assigned colors are Red = F2100W + F1130W + F1000W + F770W, Green = F770W + F360M, Blue = F335M + F300M. For the HST part of the image, the assigned colors are Red = F814W + F656N, Green = F555W, Blue = F435W. Overlaid as contours is the PHANGS-HST Nebula Catalogue (as in the other panels). (*fig_maps*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.11778"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcidicon}[1]{\href{https://orcid.org/#1}{\includegraphics[width=11pt]{Plot/ORCIDiD_icon128x128.png}}}$
$\newcommand{\orcid}[1]{\href{https://orcid.org/#1}{\protect\orcidicon{#1}}}$
$\newcommand{\msun}{{\rm M}_\odot}$
$\newcommand{\ST}[1]{\textcolor{steelblue!100}{#1_{\mathrm{ST}}}}$</div>



<div id="title">

# Optically thick winds of very massive stars \ suppress intermediate-mass black hole formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.12465-b31b1b.svg)](https://arxiv.org/abs/2510.12465)<mark>Appeared on: 2025-10-15</mark> -  _10 pages, 7 figures. Comments welcome_

</div>
<div id="authors">

<mark>S. Torniamenti</mark>, et al.

</div>
<div id="abstract">

**Abstract:** Intermediate-mass black holes (IMBHs) are the link between stellar-mass and supermassive black holes.Gravitational waves havestarted unveiling a population of IMBHs in the $\sim 100-300   \msun$ range.Here, we investigate the formation of IMBHs from very massive stars (VMSs, $>100   \msun$ ). We calculate new VMS models that account for the transition from optically thin to optically thick winds, and study how this enhanced mass loss affects  IMBH formation and the black hole mass function at intermediate and high metallicity ( $Z=10^{-4}-0.02$ ).We show that optically thick winds suppress the formation of IMBHs from direct VMS collapse at metallicities $Z>0.001$ , one order of magnitude lower than  predicted by previous models.Our models indicate that the stellar progenitors of GW231123 must have had a metallicity $Z<0.002$ , if the primary black hole formed via direct VMS collapse.

</div>

<div id="div_fig1">

<img src="tmp_2510.12465/./Plot/MBH_Z_mesa_parsec_v_log.png" alt="Fig22" width="100%"/>

**Figure 22. -** Final BH mass as a function of $M_{\mathrm{ZAMS}}$ for different metallicities $Z$,  for the \citetalias{Sabhahit2023}(upper) and \citetalias{Chen2015}(lower)  wind mass-loss prescription. The grey shaded area shows the resulting pair-instability mass gap for the two wind models. (*fig:MBH_Z*)

</div>
<div id="div_fig2">

<img src="tmp_2510.12465/./Plot/m1_merged_bin_Z.png" alt="Fig16" width="100%"/>

**Figure 16. -** Primary BH mass distribution in BBH mergers, for models adopting the \citetalias{Sabhahit2023}(blue, solid line) \citetalias{Chen2015}(orange, dashed line) wind prescriptions. The shaded (hatched) area shows the BH masses from stars with $M_{\mathrm{ZAMS}} > 50 $\msun$$ for the \citetalias{Sabhahit2023}(\citetalias{Chen2015}), respectively. (*fig:bin_BH_pop*)

</div>
<div id="div_fig3">

<img src="tmp_2510.12465/./Plot/mass_evolution.png" alt="Fig17" width="100%"/>

**Figure 17. -** Mass evolution for models with the wind prescription from Sabhahit+23 (full markers) and Chen+15 (void markers). MODELS with Chen+15 evolved with SEVN based on PARSEC tracks. \textcolor{red}{NOTE THAT models with Z=0.02 and Z=0.008 actually use Sabhahit+22, which was conceived for higher metallicities.} (*fig:BH_sevn*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.12465"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

122  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
